# Import packages

In [ ]:
import statsmodels.api as sm
import pandas as pd
import numpy as np

Note: Controlling outputs of DataFrame - [pd.Options()](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html)

In [ ]:
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)

# Dataset - nycflights13

Honestly speaking... Python pandas is lack of built-in dataset for demostration use.

We use the function in statsmodels - [get_rdataset](https://www.statsmodels.org/dev/datasets/statsmodels.datasets.get_rdataset.html#statsmodels.datasets.get_rdataset) for obtaining [R datasets](https://vincentarelbundock.github.io/Rdatasets/index.html) .

In this colab we use the dataset for teaching data propcessing package [tidyverse](https://www.tidyverse.org/) - which is called [nycflight13](https://nycflights13.tidyverse.org/)

![](https://drive.google.com/uc?export=view&id=14JryRx2tlzlt_ZuqmT3YuZaMF0FGsHIK)

This package contains information about all flights that departed from NYC (e.g. EWR, JFK and LGA) to destinations in the United States in 2013.

In [ ]:
airlines = sm.datasets.get_rdataset('airlines', 'nycflights13').data
airports = sm.datasets.get_rdataset('airports', 'nycflights13').data
flights = sm.datasets.get_rdataset('flights', 'nycflights13').data
planes = sm.datasets.get_rdataset('planes', 'nycflights13').data
weather = sm.datasets.get_rdataset('weather', 'nycflights13').data

In [ ]:
print(flights)

or just import csv with the url

In [ ]:
flights_local = pd.read_csv('https://vincentarelbundock.github.io/Rdatasets/csv/nycflights13/flights.csv')

Input & Output (IO), not only can do csv format, but also json, xlsx, parquet (much more convenient than csv), and clipboard... Please refer to [pandas document](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html).

# Skim the data

Consider the flights dataset.

In [ ]:
flights.info()

In [ ]:
flights.shape

In [ ]:
flights.head()

In [ ]:
print(flights.__doc__)

In [ ]:
flights.describe()

# Handling Missing Values

Removing rows: [dropna](https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.DataFrame.dropna.html)


Filling NA values: [fillna](https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.DataFrame.fillna.html) <br> with method parameter bfill, ffill


Imputation - Filling NA values with mean, median, ... : [sklearn.impute.SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)


Please depend to your need for choosing suitable strategies for handling missing value.

In [ ]:
flights.dropna(inplace=True)

In [ ]:
flights.info()

# Filtering

Stardard filtering using [ ].

In [ ]:
df_jan1 = flights[(flights.month == 1) & (flights.day == 1)]
df_jan1

great function [query](https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.DataFrame.query.html). Welcome to read the syntex and example in the document.

In [ ]:
df_dec25 = flights.query('(month == 12) & (day == 25)')
df_dec25

In [ ]:
flights.query('month.isin([11, 12])', engine='python')

Reference: pandas [query](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html)

# Sorting

[sort_values](https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.DataFrame.sort_values.html) function

In [ ]:
flights.sort_values(['year', 'month', 'day'])

In [ ]:
flights.sort_values(['dep_delay'],  ascending=False).reset_index()

# Selecting Column

In [ ]:
flights[['year', 'month', 'day']]

In [ ]:
flights.filter(regex='time$', axis=1)

In [ ]:
flights.drop(['year', 'month', 'day'], axis=1)
# filter.drop(column=['year', 'month', 'day'])

In [ ]:
flights[['time_hour', 'air_time'] + list(flights.columns)]
# select time_hour, air_time, *

# Rename Column

In [ ]:
# flights.rename({'tailnum': 'tail_num'}, axis=1)
flights.rename(columns = {'tailnum': 'tail_num'})


# New Column
also you can call this as "Feature Engineering".

In [ ]:
flights_temp = flights[['year', 'month', 'day', 'dep_delay', 'arr_delay', 'distance', 'air_time']]
flights_temp['gain'] = flights_temp.dep_delay - flights_temp.arr_delay
flights_temp['speed'] = flights_temp.distance / flights_temp.air_time * 60

In [ ]:
flights_temp.head()

or you can use dataframe method [assign](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html).

In [ ]:
flights_temp = flights[['year', 'month', 'day', 'dep_delay', 'arr_delay', 'distance', 'air_time']]
flights_temp.assign(gain=lambda x: x.dep_delay - x.arr_delay
                    , speed=lambda x: x.distance / x.air_time * 60
                    )

also you can chain methods according to your processing step.

In [ ]:
flights_temp = flights[['year', 'month', 'day', 'dep_delay', 'arr_delay', 'distance', 'air_time']]
flights_temp.assign(gain=lambda x: x.dep_delay - x.arr_delay
                    , hours=lambda x: x.air_time / 60
                    ) \
              .assign(gain_per_hour=lambda x: x.gain / x.hours)

# Group-by summary measures

In [ ]:
flights.dep_delay.mean()

In [ ]:
flights.groupby(['year', 'month', 'day']).dep_delay.mean() \
      .reset_index()

# Chain methods to be pipeline

Firstly we show the non-pipeline version.

In [ ]:
by_dest = flights.groupby('dest')
delay = by_dest.agg(count=('dest', 'count')
    , dist=('distance', 'mean')
    , delay=('arr_delay', 'mean')
    )
delay = delay.query('(count > 20) & (dest != "HNL")')

delay.sort_values('delay', ascending = False).head()

select count(dest) as count, mean(distance) as dist, mean(arr_delay) as delay <br>
from by_dest <br>
group by dest <br>

On the other hand, pipeline version.

In [ ]:
delay = flights.groupby('dest') \
    .agg(count=('dest', 'count')
        , dist=('distance', 'mean')
        , delay=('arr_delay', 'mean')
        ) \
    .query('(count > 20) & (dest != "HNL")')

delay.head()

# Merging, or Joining

Firstly, investigating airlines, airports, planes, weather datasets.

In [ ]:
airlines.info()
airlines.head()

In [ ]:
airports.info()
airports.head()

In [ ]:
planes.info()
planes.head()

In [ ]:
weather.info()
weather.head()

### Checking Primary Key

In [ ]:
planes.tailnum.value_counts()

In [ ]:
# planes key = tailnum
temp = planes.tailnum.value_counts()
temp[temp > 1]

In [ ]:
# weather key = year, month, day, hour, origin
temp = weather[['year', 'month', 'day', 'hour', 'origin']].value_counts()
temp[temp > 1] # huh?

In [ ]:
# Why ?
weather.query('(year == 2013) & (month == 11) & (day == 3) & (hour == 1) & (origin == "EWR")')

### Join Example

[merge](https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.DataFrame.merge.html) function

In [ ]:
flights2 = flights[['year', 'month', 'day', 'hour', 'origin', 'dest', 'tailnum', 'carrier']]

In [ ]:
flights2.drop(columns=['origin', 'dest']).merge(airlines, how='left', on='carrier')

how ? left, right, inner, outer

In [ ]:
x = pd.DataFrame({'key': [1, 2, 3], 'val_x': ['x1', 'x2', 'x3']})
y = pd.DataFrame({'key': [1, 2, 4], 'val_y': ['y1', 'y2', 'y3']})
print(x)
print(y)

In [ ]:
print(x.merge(y, on='key')) # inner
print(x.merge(y, how='outer', on='key')) # outer
print(x.merge(y, how='left', on='key')) # left
print(x.merge(y, how='right', on='key')) # right

Back to Join Example

In [ ]:
flights2

In [ ]:
weather

In [ ]:
flights2.merge(weather, how='left') # Omitting "on" parameter takes overlapping column names

In [ ]:
flights2.merge(planes, how='left', on="tailnum")

In [ ]:
flights2.merge(airports, how='left', left_on="dest", right_on="faa")